In [82]:
import findspark

In [83]:

findspark.init("/opt/manual/spark")

In [84]:

from pyspark.sql import SparkSession, functions as F
import pandas as pd

In [85]:
# https://github.com/Azure/mmlspark   ## LİGHTGBM KULLANIMI
#spark = (SparkSession.builder
        #.appName("HChurn Prediction with LightGBM on Spark")
        #.master("yarn")
         #.config("spark.memory.fraction","0.7")
         #.config("spark.memory.storageFraction","0.3")
         #.config("spark.executor.cores","2") # lightgbm requires at least 2 cores per task      
         #.config("spark.jars.packages","Azure:mmlspark:0.17")
        #.enableHiveSupport()
        #.getOrCreate())

In [86]:
from mmlspark.lightgbm import LightGBMRegressor

ModuleNotFoundError: No module named 'mmlspark'

In [87]:
spark = SparkSession.builder \
.appName("arabam_com") \
.master("yarn") \
.getOrCreate()

In [88]:
#! hdfs dfs -rm /user/train/datasets/arabam.csv

In [89]:
#! hdfs dfs -put ~/Desktop/arabam/arabam.csv /user/train/datasets/

In [90]:
df1 = spark.read.format("csv") \
.option("header",True) \
.option("inferShema",True) \
.option("sep",",")  \
.load("hdfs://localhost:9000/user/train/datasets/arabam.csv")

In [91]:
df1.limit(10).toPandas()

,_c0,İlan No:,İlan Tarihi:,Marka:,Seri:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Motor Hacmi:,Motor Gücü:,Kilometre:,car_link:,price
0,0,16265968,25 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Distinctive,2014,Dizel,Düz,1598 cc,105 hp,82.000 km,https://www.arabam.com//ilan/sahibinden-satili...,192.500 TL
1,1,16265262,25 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Progression Plus,2012,Dizel,Düz,1598 cc,105 hp,165.500 km,https://www.arabam.com//ilan/sahibinden-satili...,125.000 TL
2,2,16262696,24 Aralık 2020,Alfa Romeo,156,2.0 JTS Distinctive,2003,Benzin,Yarı Otomatik,1801 - 2000 cm3,151 - 175 HP,191.000 km,https://www.arabam.com//ilan/galeriden-satilik...,77.000 TL
3,3,16129020,24 Aralık 2020,Alfa Romeo,Giulietta,1.4 TB MultiAir Distinctive,2015,LPG & Benzin,Yarı Otomatik,1201 - 1400 cm3,151 - 175 HP,117.000 km,https://www.arabam.com//ilan/galeriden-satilik...,194.000 TL
4,4,16259218,24 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Distinctive,2014,Dizel,Düz,1598 cc,105 hp,66.400 km,https://www.arabam.com//ilan/sahibinden-satili...,180.000 TL
5,5,16258876,24 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Distinctive,2014,Dizel,Düz,1598 cc,105 hp,113.000 km,https://www.arabam.com//ilan/sahibinden-satili...,178.000 TL
6,6,16257484,23 Aralık 2020,Alfa Romeo,156,1.6 TS,1998,LPG & Benzin,Düz,1598 cc,120 hp,220.000 km,https://www.arabam.com//ilan/sahibinden-satili...,45.950 TL
7,7,16255409,23 Aralık 2020,Alfa Romeo,145,1.6,1997,LPG & Benzin,Düz,1596 cc,103 hp,350.000 km,https://www.arabam.com//ilan/galeriden-satilik...,36.750 TL
8,8,16252962,23 Aralık 2020,Alfa Romeo,156,1.6 TS Progression,2004,LPG & Benzin,Düz,1401 - 1600 cm3,101 - 125 HP,175.000 km,https://www.arabam.com//ilan/sahibinden-satili...,66.000 TL
9,9,15289709,23 Aralık 2020,Alfa Romeo,Giulietta,1.4 TB Progression Plus,2011,Benzin,Düz,1368 cc,120 hp,247.000 km,https://www.arabam.com//ilan/sahibinden-satili...,127.000 TL


In [92]:
df1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- İlan No:: string (nullable = true)
 |-- İlan Tarihi:: string (nullable = true)
 |-- Marka:: string (nullable = true)
 |-- Seri:: string (nullable = true)
 |-- Model:: string (nullable = true)
 |-- Yıl:: string (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- Motor Hacmi:: string (nullable = true)
 |-- Motor Gücü:: string (nullable = true)
 |-- Kilometre:: string (nullable = true)
 |-- car_link:: string (nullable = true)
 |-- price: string (nullable = true)



In [93]:
df1.columns

['_c0',
 'İlan No:',
 'İlan Tarihi:',
 'Marka:',
 'Seri:',
 'Model:',
 'Yıl:',
 'Yakıt Tipi:',
 'Vites Tipi:',
 'Motor Hacmi:',
 'Motor Gücü:',
 'Kilometre:',
 'car_link:',
 'price']

In [94]:
for col in df1.columns:
    df1 = df1.withColumn(col, F.ltrim(F.rtrim(df1[col])))

In [95]:
df2 =df1.withColumn("motor_hacmi",F.split('Motor Hacmi:',"-")[0]) \
.withColumn("price",F.split("price"," ")[0]) \
.withColumn("kilometre",F.split("Kilometre:"," ")[0]) \
.drop(F.col('Motor Hacmi:')) 

 


In [96]:
df2=df2.dropna(how="any")

In [97]:
#.withColumn("price",F.split("price",".")[0:2]) \
#.withColumn("kilometre",F.split("Kilometre:",".")[0:2]) \

In [98]:
df3 =df2.withColumn("motor_hacmi",F.split("motor_hacmi"," ")[0]) \
.withColumn("motor_gucu",F.split("Motor Gücü:"," ")[0]) \
.withColumn("marka",F.concat(F.col("Marka:"),F.lit('_'),F.col("Seri:"),F.lit('_'),F.col("Model:"))) \


In [99]:
df3.limit(2).toPandas()

,_c0,İlan No:,İlan Tarihi:,Marka:,Seri:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Motor Gücü:,Kilometre:,car_link:,price,motor_hacmi,kilometre,motor_gucu,marka
0,0,16265968,25 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Distinctive,2014,Dizel,Düz,105 hp,82.000 km,https://www.arabam.com//ilan/sahibinden-satili...,192.500,1598,82.000,105,Alfa Romeo_Giulietta_1.6 JTD Distinctive
1,1,16265262,25 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Progression Plus,2012,Dizel,Düz,105 hp,165.500 km,https://www.arabam.com//ilan/sahibinden-satili...,125.000,1598,165.500,105,Alfa Romeo_Giulietta_1.6 JTD Progression Plus


In [100]:
df4=df3.withColumn("price1",F.split(F.col("price"),"\.").getItem(0)) \
.withColumn("price2",F.split(F.col("price"),"\.").getItem(1)) \
.withColumn("price",F.concat(F.col("price1"),F.col("price2")))

In [101]:
df5=df4.withColumn("km1",F.split(F.col("kilometre"),"\.").getItem(0)) \
.withColumn("km2",F.split(F.col("kilometre"),"\.").getItem(1)) \
.withColumn("km",F.concat(F.col("km1"),F.col("km2")))

In [102]:
df6=df5.drop(F.col("km1")) \
.drop(F.col("km2")) \
.drop(F.col("price1")) \
.drop(F.col("price2"))

In [103]:
df6.limit(2).toPandas()

,_c0,İlan No:,İlan Tarihi:,Marka:,Seri:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Motor Gücü:,Kilometre:,car_link:,price,motor_hacmi,kilometre,motor_gucu,marka,km
0,0,16265968,25 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Distinctive,2014,Dizel,Düz,105 hp,82.000 km,https://www.arabam.com//ilan/sahibinden-satili...,192500,1598,82.000,105,Alfa Romeo_Giulietta_1.6 JTD Distinctive,82000
1,1,16265262,25 Aralık 2020,Alfa Romeo,Giulietta,1.6 JTD Progression Plus,2012,Dizel,Düz,105 hp,165.500 km,https://www.arabam.com//ilan/sahibinden-satili...,125000,1598,165.500,105,Alfa Romeo_Giulietta_1.6 JTD Progression Plus,165500


In [104]:
df6.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- İlan No:: string (nullable = true)
 |-- İlan Tarihi:: string (nullable = true)
 |-- Marka:: string (nullable = true)
 |-- Seri:: string (nullable = true)
 |-- Model:: string (nullable = true)
 |-- Yıl:: string (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- Motor Gücü:: string (nullable = true)
 |-- Kilometre:: string (nullable = true)
 |-- car_link:: string (nullable = true)
 |-- price: string (nullable = true)
 |-- motor_hacmi: string (nullable = true)
 |-- kilometre: string (nullable = true)
 |-- motor_gucu: string (nullable = true)
 |-- marka: string (nullable = true)
 |-- km: string (nullable = true)



In [105]:
df7=df6.select(
F.col('Yıl:').cast("int"),
F.col('Yakıt Tipi:'),
F.col('Vites Tipi:'),
F.col('km').cast("int"),
F.col("price").cast("int"),
F.col("car_link:"),
F.col("marka"),
F.col("Seri:"))

In [106]:
df7.printSchema()

root
 |-- Yıl:: integer (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- km: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- car_link:: string (nullable = true)
 |-- marka: string (nullable = true)
 |-- Seri:: string (nullable = true)



In [107]:
df7.limit(3).toPandas()

,Yıl:,Yakıt Tipi:,Vites Tipi:,km,price,car_link:,marka,Seri:
0,2014,Dizel,Düz,82000,192500,https://www.arabam.com//ilan/sahibinden-satili...,Alfa Romeo_Giulietta_1.6 JTD Distinctive,Giulietta
1,2012,Dizel,Düz,165500,125000,https://www.arabam.com//ilan/sahibinden-satili...,Alfa Romeo_Giulietta_1.6 JTD Progression Plus,Giulietta
2,2003,Benzin,Yarı Otomatik,191000,77000,https://www.arabam.com//ilan/galeriden-satilik...,Alfa Romeo_156_2.0 JTS Distinctive,156


In [108]:
# Null check
for(col_name, col_type) in zip(df7.columns, df7.dtypes):
    null_count = df7.filter( (F.col(col_name).isNull()) | (F.col(col_name) == "")).count()
    if(  null_count > 0 ):
        print("{} {} type has {} null values".format(col_name, col_type[1], null_count))

km int type has 568 null values


In [109]:
df8=df7.dropna(how="any")

In [110]:

categoric_cols = []
numeric_cols = []
discarted_cols = []
label_col = []

In [111]:
# column seperation
for col_name in df8.dtypes:
    if (col_name[0] not in []):
        if (col_name[1] == 'string'):
            categoric_cols.append(col_name[0])
        else: numeric_cols.append(col_name[0])

In [112]:
for col_name in categoric_cols:
    df8.groupBy(col_name).agg(F.count("*").alias("TotalCount")).orderBy(F.desc("TotalCount")).show()

+------------+----------+
| Yakıt Tipi:|TotalCount|
+------------+----------+
|       Dizel|      9585|
|LPG & Benzin|      7079|
|      Benzin|      5202|
|      Hibrit|         7|
|    Elektrik|         3|
+------------+----------+

+-------------+----------+
|  Vites Tipi:|TotalCount|
+-------------+----------+
|          Düz|     12836|
|Yarı Otomatik|      4722|
|     Otomatik|      4318|
+-------------+----------+

+--------------------+----------+
|           car_link:|TotalCount|
+--------------------+----------+
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         1|
|https://www.araba...|         

In [113]:
df8.columns

['Yıl:',
 'Yakıt Tipi:',
 'Vites Tipi:',
 'km',
 'price',
 'car_link:',
 'marka',
 'Seri:']

In [114]:

# column seperation after analysis
categoric_cols = []
numeric_cols = []
# Why we discard these? Because they have high cardinality. In other words they have too many distinct categories
# Adding them to model will not do good if not too bad.
discarted_cols = ['car_link:']
label_col = ['price']

In [115]:

for col_name in df8.dtypes:
    if (col_name[0] not in discarted_cols+label_col):
        if (col_name[1] == 'string'):
            categoric_cols.append(col_name[0])
        else: numeric_cols.append(col_name[0])

In [116]:
# Here we count the distinct categories in categoric columns. If there is more than two,
# we will add those to to_be_onehotencoded_cols list If there is just 2 we don't need to use onehotencoder
# So if distinct category gt 2 we have to add it to_be_onehotencoded_cols list
to_be_onehotencoded_cols = []

for col_name in categoric_cols:
    count = df8.select(col_name).distinct().count()
    if count > 2:
        to_be_onehotencoded_cols.append(col_name)
    print("{} has {} distinct category.".format(col_name, count))
    

Yakıt Tipi: has 5 distinct category.
Vites Tipi: has 3 distinct category.
marka has 2998 distinct category.
Seri: has 403 distinct category.


In [117]:
print(to_be_onehotencoded_cols)

['Yakıt Tipi:', 'Vites Tipi:', 'marka', 'Seri:']


# StringIndexer

In [118]:
from pyspark.ml.feature import StringIndexer

In [119]:
# Will hold stringIndexer objects and column names
my_dict = {}

# Will collect StringIndexer ojects
string_indexer_objs = []

# Will collect StringIndexer output colnames
string_indexer_output_names = []

# Will collect OneHotEncoder output colnames
ohe_col_input_names = []
ohe_col_output_names = []

for col_name in categoric_cols:
    my_dict[col_name+"_indexobj"] = StringIndexer() \
    .setHandleInvalid('skip') \
    .setInputCol(col_name) \
    .setOutputCol(col_name+"_indexed")
    
    string_indexer_objs.append(my_dict.get(col_name+"_indexobj"))
    string_indexer_output_names.append(col_name+"_indexed")
    if col_name in to_be_onehotencoded_cols:
        ohe_col_input_names.append(col_name+"_indexed")
        ohe_col_output_names.append(col_name+"_ohe")

In [120]:
print(string_indexer_objs)
print(len(string_indexer_objs))

[StringIndexer_2e6e36f5809a, StringIndexer_b5ae843d899e, StringIndexer_1c1dd820d6f4, StringIndexer_48d04322b490]
4


In [121]:
print(ohe_col_input_names)
print(len(ohe_col_input_names))

['Yakıt Tipi:_indexed', 'Vites Tipi:_indexed', 'marka_indexed', 'Seri:_indexed']
4


In [122]:
print(string_indexer_output_names)
print(len(string_indexer_output_names))

['Yakıt Tipi:_indexed', 'Vites Tipi:_indexed', 'marka_indexed', 'Seri:_indexed']
4


# OnehotEncoder

In [123]:
from pyspark.ml.feature import OneHotEncoder

In [124]:
encoder = OneHotEncoder() \
.setInputCols(ohe_col_input_names) \
.setOutputCols(ohe_col_output_names)

In [125]:
from pyspark.ml.feature import VectorAssembler

## VectorAssembler

In [126]:
string_indexer_col_names_ohe_exluded = list(set(string_indexer_output_names).difference(set(ohe_col_input_names)))
string_indexer_col_names_ohe_exluded

[]

In [127]:
if len(categoric_cols) == (len(string_indexer_col_names_ohe_exluded)+len(ohe_col_output_names)):
    print("Columns verified")
else: print("Columns are not verified")

Columns verified


In [128]:
assembler = VectorAssembler().setHandleInvalid("skip") \
.setInputCols(numeric_cols+string_indexer_col_names_ohe_exluded+ohe_col_output_names) \
.setOutputCol('unscaled_features')

In [129]:
label_col

['price']

In [130]:
from pyspark.ml.feature import StandardScaler

In [131]:
scaler = StandardScaler().setInputCol("unscaled_features").setOutputCol("scaled_features")

## PCA

In [132]:
from pyspark.ml.feature import PCA

In [133]:
#pca = PCA().setInputCol("scaled_features").setK(10).setOutputCol("features")

# Estimator: LinearRegression

In [134]:
from pyspark.ml.regression import LinearRegression


In [135]:
estimator_linear = LinearRegression() \
.setFeaturesCol("scaled_features") \
.setLabelCol("price")

# Estimator: GBTRegression

In [136]:
from pyspark.ml.regression import GBTRegressor

In [137]:
estimator_GBT=GBTRegressor() \
.setFeaturesCol("scaled_features") \
.setLabelCol("price")

# Estimator: LightgbmRegressor

In [57]:

from mmlspark.lightgbm import LightGBMRegressor

ModuleNotFoundError: No module named 'mmlspark'

In [ ]:
model = LightGBMRegressor(objective='quantile',
                          alpha=0.2,
                          learningRate=0.3,
                          numLeaves=31)

#  Estimator: RandomForest

In [138]:
from pyspark.ml.regression import RandomForestRegressor

In [139]:
estimator_RandomForest = RandomForestRegressor() \
.setFeaturesCol("scaled_features") \
.setLabelCol("price")


## Pipeline

In [140]:
obj_list=[estimator_RandomForest,estimator_GBT,estimator_linear]

In [141]:
from pyspark.ml import Pipeline

In [62]:
pipeline_obj = Pipeline().setStages(string_indexer_objs+[encoder, assembler, scaler, estimator_RandomForest])


# Split data

In [142]:
train_df, test_df = df8.randomSplit([0.80,0.10], seed=142)

In [143]:
train_df.count(),test_df.count()

(19430, 2446)

# Train MoDEL

In [65]:
pipeline_model = pipeline_obj.fit(train_df)

# Save the model

In [176]:
pipeline_model.stages[-1].write().overwrite().save("~/Desktop/arabam/randomForestmodel")

# Prediction

In [69]:
transformed_df = pipeline_model.transform(test_df)

In [70]:
transformed_df.select('price','prediction').show(truncate=False)

+-----+------------------+
|price|prediction        |
+-----+------------------+
|45000|75524.91369594776 |
|45000|104867.20418653838|
|66000|79713.24130845729 |
|10500|54989.06803951076 |
|30000|95470.89045144807 |
|65000|182017.09319988932|
|20000|54989.06803951076 |
|28500|42925.14117179888 |
|18000|42925.14117179888 |
|21950|42925.14117179888 |
|20500|54989.06803951076 |
|40500|98742.95275730538 |
|66700|104867.20418653838|
|34750|97494.90154985644 |
|29500|52258.31272601751 |
|24000|53475.68723910172 |
|38000|53475.68723910172 |
|20100|44142.51568488308 |
|46000|54989.06803951076 |
|56250|58046.303429914595|
+-----+------------------+
only showing top 20 rows



# Evaluate the model

In [144]:
from pyspark.ml.evaluation import RegressionEvaluator

In [145]:
evaluator = RegressionEvaluator(labelCol='price')

In [146]:
evaluator.evaluate(transformed_df)

Py4JJavaError: An error occurred while calling o3204.evaluate.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)


In [147]:
evaluator.getMetricName()

'rmse'

## compare models

In [148]:
model_list=[estimator_RandomForest,estimator_GBT,estimator_linear]
for model_obj in model_list:
    model=Pipeline().setStages(string_indexer_objs+[encoder, assembler, scaler, model_obj]).fit(train_df)
    transformed_df = model.transform(test_df)
    evaluator = RegressionEvaluator(labelCol='price')
    evaluator.evaluate(transformed_df)
    evaluator.getMetricName()
    print("model name : {} metrik {} : {} ".format("model_"+str(model_obj),evaluator.getMetricName(),evaluator.evaluate(transformed_df)))

model name : model_RandomForestRegressor_5e6169ef1c88 metrik rmse : 66878.08015322544 
model name : model_GBTRegressor_dee3d25ee70c metrik rmse : 52907.760533007095 
model name : model_LinearRegression_d4859fc83a1a metrik rmse : 46810.365265321285 


In [462]:
train_df.count(),test_df.count()

(19430, 2446)

# Hyperparameter Tuning

In [149]:
from pyspark.ml.tuning import CrossValidatorModel, ParamGridBuilder, CrossValidator

In [150]:
paramGrid = ParamGridBuilder() \
.addGrid(estimator_linear.regParam, [0.01,0.1,1.0,10.0]) \
.addGrid(estimator_linear.elasticNetParam,[0.0,0.5,1.0]) \
.build()

In [151]:
pipeline_obj = Pipeline().setStages(string_indexer_objs+[encoder, assembler, scaler, estimator_linear])

In [152]:
crossval = CrossValidator(estimator=pipeline_obj,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol='price'),
                          numFolds=5) 

In [153]:
cvModel = crossval.fit(df8)

KeyboardInterrupt: 

In [ ]:
type(cvModel)

In [ ]:
cvModel.params

In [ ]:
bestModel = cvModel.bestModel

In [ ]:
evaluator.evaluate(bestModel.transform(df8))

In [ ]:
evaluator.evaluate(cvModel.transform(df8))

In [553]:
bestModel.write().overwrite().save("~/Desktop/arabam/arabam_best_model_linear")

In [81]:
spark.stop()